In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 설정
BASE_URL = 'https://m.dcinside.com'
GALLERY_ID = 'programming'  # 예시: 프로그래밍 갤러리 (네가 원하는 걸로 바꿔)
MAX_POSTS = 100
MAX_COMMENTS_PER_POST = 50

# 결과 저장
posts = []

# Step 1: 인기글 리스트 가져오기
def get_popular_posts(gallery_id):
    url = f'{BASE_URL}/board/{gallery_id}?recommend=1'  # 개념글 목록
    res = requests.get(url)
    soup = BeautifulSoup(res.text, 'html.parser')
    links = soup.select('tr.ub-content > td.gall_tit > a')
    post_links = [BASE_URL + link['href'] for link in links]
    return post_links

# Step 2: 게시글 본문 + 댓글 가져오기
def get_post_details(post_url):
    res = requests.get(post_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    try:
        title = soup.select_one('div.ub-content > div.title_subject').text.strip()
    except:
        title = ""

    try:
        body = soup.select_one('div.write_div').text.strip()
    except:
        body = ""

    try:
        like_count = int(soup.select_one('span.up_num').text.strip())
    except:
        like_count = 0

    try:
        comment_count = int(soup.select_one('span.cmt_count').text.strip().replace('댓글', '').strip())
    except:
        comment_count = 0

    try:
        time_tag = soup.select_one('div.gall_date')['title']
    except:
        time_tag = ""

    post_id = post_url.split('/')[-1]
    
    comments = get_comments(post_id)
    
    return {
        'post_id': post_id,
        'title': title,
        'body': body,
        'like_count': like_count,
        'comment_count': comment_count,
        'time': time_tag,
        'link': post_url,
        'comments': comments
    }

# Step 3: 댓글 가져오기
def get_comments(post_id):
    comments = []
    page = 1

    while len(comments) < MAX_COMMENTS_PER_POST:
        comment_url = f"https://m.dcinside.com/board/comment/{post_id}?page={page}"
        res = requests.get(comment_url)
        soup = BeautifulSoup(res.text, 'html.parser')
        comment_list = soup.select('ul.comment_box > li')
        
        if not comment_list:
            break  # 더 이상 댓글 없음
        
        for comment in comment_list:
            content_tag = comment.select_one('p.usertxt')
            if content_tag:
                content = content_tag.text.strip()
                comments.append(content)
            
            if len(comments) >= MAX_COMMENTS_PER_POST:
                break
        
        page += 1
        time.sleep(0.1)  # 서버 부하 줄이기

    return comments

# 메인 크롤링
def crawl_dc_popular(gallery_id):
    post_links = get_popular_posts(gallery_id)
    print(f"Found {len(post_links)} popular posts.")
    
    for i, link in enumerate(post_links[:MAX_POSTS]):
        print(f"[{i+1}/{MAX_POSTS}] 크롤링 중: {link}")
        post = get_post_details(link)
        posts.append(post)
        time.sleep(0.5)  # 서버 부하 줄이기

    # CSV 저장
    df = pd.DataFrame(posts)
    df.to_csv(f'dc_{gallery_id}_popular.csv', index=False, encoding='utf-8-sig')
    print("CSV 저장 완료")

# 실행
if __name__ == "__main__":
    crawl_dc_popular(GALLERY_ID)


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# 설정
BASE_URL = 'https://gall.dcinside.com'
MAIN_POPULAR_URL = f'{BASE_URL}/board/lists?id=all&exception_mode=recommend'
MAX_POSTS = 100
MAX_COMMENTS_PER_POST = 50

# 결과 저장
posts = []

# Step 1: 메인 인기글 목록 가져오기
def get_main_popular_posts():
    res = requests.get(MAIN_POPULAR_URL)
    soup = BeautifulSoup(res.text, 'html.parser')
    links = soup.select('tr.ub-content > td.gall_tit > a')
    post_links = [BASE_URL + link['href'] for link in links if '/board/view/' in link['href']]
    return post_links

# Step 2: 게시글 본문 + 댓글 가져오기
def get_post_details(post_url):
    res = requests.get(post_url)
    soup = BeautifulSoup(res.text, 'html.parser')
    
    try:
        title = soup.select_one('span.title_subject').text.strip()
    except:
        title = ""

    try:
        body = soup.select_one('div.write_div').text.strip()
    except:
        body = ""

    try:
        like_count = int(soup.select_one('span.up_num').text.strip())
    except:
        like_count = 0

    try:
        comment_count = int(soup.select_one('span.cmt_count').text.strip().replace('댓글', '').strip())
    except:
        comment_count = 0

    try:
        time_tag = soup.select_one('span.gall_date').text.strip()
    except:
        time_tag = ""

    post_id = post_url.split('no=')[-1]
    
    comments = get_comments(post_id)
    
    return {
        'post_id': post_id,
        'title': title,
        'body': body,
        'like_count': like_count,
        'comment_count': comment_count,
        'time': time_tag,
        'link': post_url,
        'comments': comments
    }

# Step 3: 댓글 가져오기
def get_comments(post_id):
    comments = []
    page = 1

    while len(comments) < MAX_COMMENTS_PER_POST:
        comment_url = f"https://gall.dcinside.com/board/comment/lists/?id=all&no={post_id}&comment_page={page}"
        res = requests.get(comment_url)
        soup = BeautifulSoup(res.text, 'html.parser')
        comment_list = soup.select('ul.comment_box > li')
        
        if not comment_list:
            break  # 더 이상 댓글 없음
        
        for comment in comment_list:
            content_tag = comment.select_one('p.usertxt')
            if content_tag:
                content = content_tag.text.strip()
                comments.append(content)
            
            if len(comments) >= MAX_COMMENTS_PER_POST:
                break
        
        page += 1
        time.sleep(0.1)  # 서버 부하 방지

    return comments

# 메인 크롤링
def crawl_dc_main_popular():
    post_links = get_main_popular_posts()
    print(f"Found {len(post_links)} popular posts.")

    for i, link in enumerate(post_links[:MAX_POSTS]):
        print(f"[{i+1}/{MAX_POSTS}] 크롤링 중: {link}")
        post = get_post_details(link)
        posts.append(post)
        time.sleep(0.5)  # 서버 부하 방지

    # CSV 저장
    df = pd.DataFrame(posts)
    df.to_csv(f'dc_main_popular.csv', index=False, encoding='utf-8-sig')
    print("CSV 저장 완료!")

# 실행
if __name__ == "__main__":
    crawl_dc_main_popular()


Found 0 popular posts.
CSV 저장 완료!


In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd

# 설정
BASE_URL = "https://gall.dcinside.com"
SILBE_URL = f"{BASE_URL}/board/lists?id=dcbest"
MAX_POSTS = 100
MAX_COMMENTS_PER_POST = 50
POSTS_PER_PAGE = 20

# requests 세션 생성
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Referer": "https://gall.dcinside.com/"
})

results = []

# 실베 페이지별 글 링크 가져오기
def get_post_links(page_num):
    url = f"{SILBE_URL}&page={page_num}"
    res = session.get(url)
    soup = BeautifulSoup(res.text, "html.parser")
    links = []

    # 'tr' 중에 class가 'ub-content us-post'인 것만 선택
    posts = soup.select('tr.ub-content.us-post')

    for post in posts:
        link_tag = post.select_one('td.gall_tit a')
        if link_tag:
            href = link_tag.get("href")
            if href and '/board/view/' in href:
                if href.startswith('http'):
                    full_link = href
                else:
                    full_link = BASE_URL + href
                links.append(full_link)
    return links


# 게시글 본문 + 댓글 가져오기
def get_post_details(post_url):
    res = session.get(post_url)
    soup = BeautifulSoup(res.text, "html.parser")
    
    try:
        title = soup.select_one("span.title_subject").text.strip()
    except:
        title = ""

    try:
        content = soup.select_one("div.write_div").text.strip()
    except:
        content = "본문 없음"

    comment_tags = soup.select("div.usertxt")
    comments = []
    for comment_tag in comment_tags:
        if len(comments) >= MAX_COMMENTS_PER_POST:
            break
        text = comment_tag.get_text(strip=True)
        if text:
            comments.append(text)

    return {
        "title": title,
        "link": post_url,
        "content": content,
        "comments": comments
    }

# 메인 크롤링 함수
def crawl_silbe():
    page = 1
    crawled_posts = 0

    while crawled_posts < MAX_POSTS:
        print(f"페이지 {page} 크롤링 중...")
        post_links = get_post_links(page)

        for post_link in post_links:
            if crawled_posts >= MAX_POSTS:
                break
            try:
                post = get_post_details(post_link)
                results.append(post)
                crawled_posts += 1
                print(f"[{crawled_posts}] {post['title']} 크롤링 완료")
                time.sleep(0.5)  # 서버 부하 방지
            except Exception as e:
                print(f"에러 발생 (패스): {e}")
                continue
        
        page += 1
        time.sleep(1)

    # DataFrame으로 저장
    df = pd.DataFrame(results)
    df.to_csv('dcinside_silbe_100posts.csv', index=False, encoding='utf-8-sig')
    print("\n✅ CSV 파일 저장 완료: dcinside_silbe_100posts.csv")

if __name__ == "__main__":
    crawl_silbe()


페이지 1 크롤링 중...
[1] 실시간베스트 갤러리 이용 안내 크롤링 완료
[2] 실시간베스트 갤러리 이용 안내 크롤링 완료
[3] 전장연, 출근길 혜화역 지하철 시위… 또 강제 퇴거당해 크롤링 완료
[4] 전장연, 출근길 혜화역 지하철 시위… 또 강제 퇴거당해 크롤링 완료
[5] 공포의 인도 남성인권 운동 크롤링 완료
[6] 공포의 인도 남성인권 운동 크롤링 완료
[7] 전라도 군산시, 백종원 더본코리아에 70억 예산 투입 크롤링 완료
[8] 전라도 군산시, 백종원 더본코리아에 70억 예산 투입 크롤링 완료
[9] 마법과 마나의 세계에서 존재할리 없는 쿠노이치.1 크롤링 완료
[10] 마법과 마나의 세계에서 존재할리 없는 쿠노이치.1 크롤링 완료
[11] 김재연 "여가부 장관 부총리로 격상, 비동간죄·차금법 통과, 탈원전" 크롤링 완료
[12] 김재연 "여가부 장관 부총리로 격상, 비동간죄·차금법 통과, 탈원전" 크롤링 완료
[13] 씹덕 행사장에서 버튜버를 알아본 사람 크롤링 완료
[14] 씹덕 행사장에서 버튜버를 알아본 사람 크롤링 완료
[15] 싱글벙글 스타크래프트 IP경쟁 근황.jpg 크롤링 완료
[16] 싱글벙글 스타크래프트 IP경쟁 근황.jpg 크롤링 완료
[17] "닦달하면 환불 없어" '황당'문자.."돈 떼이고 협박당하나" 크롤링 완료
[18] "닦달하면 환불 없어" '황당'문자.."돈 떼이고 협박당하나" 크롤링 완료
[19] 한동훈 한화 이글스 뭐냐??????ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 크롤링 완료
[20] 한동훈 한화 이글스 뭐냐??????ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 크롤링 완료
[21] 싱글벙글 입으로 호흡 하면 안되는 이유 크롤링 완료
[22] 싱글벙글 입으로 호흡 하면 안되는 이유 크롤링 완료
[23] 어청도 2일차  탐조 크롤링 완료
[24] 어청도 2일차  탐조 크롤링 완료
[25] `이재명 측근` 정진상, `대장동 민간업자 재판`서 크롤링 완료
[26] `이재명 측근` 정진상, `대장동 민간업자 재판`서 크롤링 완료
[27] 여시)오늘